In [1]:
!pip install haystack-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 6.1 MB/s eta 0:00:00


## Logging

In [2]:
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.readers import ExtractiveReader

import logging
import haystack.logging
from haystack import tracing
from haystack.tracing.logging_tracer import LoggingTracer

# Clear any existing handlers from the root logger
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Set up a basic file handler to write logs to 'log.json'
logging.basicConfig(
    level=logging.DEBUG,  # Capture all log levels
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    handlers=[
        logging.FileHandler("log.txt", mode='w'),  # Write logs to 'log.txt'
        logging.StreamHandler()  # Also output to the console for debugging
    ]
)

# Set up Haystack logging configuration
haystack.logging.configure_logging(use_json=True)

# Set Haystack logger level to DEBUG to capture all events
haystack_logger = logging.getLogger("haystack")
haystack_logger.setLevel(logging.DEBUG)

# Tracing setup
tracing.tracer.is_content_tracing_enabled = False  # To enable tracing/logging content (inputs/outputs)
tracing.enable_tracing(
    LoggingTracer(tags_color_strings={"haystack.component.input": "\x1b[1;31m", "haystack.component.name": "\x1b[1;34m"})
)

# Document setup and writing documents to the store
docs = [
    Document(content="Paris is the capital of France."),
    Document(content="Berlin is the capital of Germany."),
    Document(content="Rome is the capital of Italy."),
    Document(content="Madrid is the capital of Spain."),
]
document_store = InMemoryDocumentStore()
document_store.write_documents(docs)

# Adding a warning to verify log output
logging.warning("This is a general warning message.")
haystack_logger.debug("This is a debug message specifically for the Haystack logger.")

# Setting up retriever, reader, and pipeline
retriever = InMemoryBM25Retriever(document_store=document_store)
reader = ExtractiveReader()
reader.warm_up()

extractive_qa_pipeline = Pipeline()
extractive_qa_pipeline.add_component(instance=retriever, name="retriever")
extractive_qa_pipeline.add_component(instance=reader, name="reader")
extractive_qa_pipeline.connect("retriever.documents", "reader.documents")

# Running the pipeline
query = "What is the capital of France?"
extractive_qa_pipeline.run(
    data={"retriever": {"query": query, "top_k": 3}, "reader": {"query": query, "top_k": 2}}
)

print("Log entries saved to log.txt")


2025-02-24 15:31:32,274 - WARNING - root - This is a general warning message.
2025-02-24 15:31:32,275 - DEBUG - haystack - This is a debug message specifically for the Haystack logger.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
2025-02-24 15:31:32,816 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): huggingface.co:443
2025-02-24 15:31:33,045 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /deepset/roberta-base-squad2-distilled/resolve/main/config.json HTTP/1.1" 200 0
2025-02-24 1

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

2025-02-24 15:31:33,359 - DEBUG - filelock - Attempting to release lock 139121492132304 on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/7aea42d1fcd0acbd46d6805b80589942daaaf0ff.lock
2025-02-24 15:31:33,361 - DEBUG - filelock - Lock 139121492132304 released on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/7aea42d1fcd0acbd46d6805b80589942daaaf0ff.lock
2025-02-24 15:31:33,565 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /deepset/roberta-base-squad2-distilled/resolve/main/adapter_config.json HTTP/1.1" 404 0
2025-02-24 15:31:37,898 - DEBUG - tensorflow - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2025-02-24 15:31:38,124 - DEBUG - h5py._conv - Creating converter from 7 to 5
2025-02-24 15:31:38,125 - DEBUG - h5py._conv - Creating converter from 5 to 7
2025-02-24 15:31:38,127 - DEBUG - h5py._conv - Creating convert

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

2025-02-24 15:31:54,418 - DEBUG - filelock - Attempting to release lock 139121486146512 on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/106dc06a09bae48ef4f4a69b9510e2d7bce9ba6536728f70ca779cdf1b2f6db3.lock
2025-02-24 15:31:54,420 - DEBUG - filelock - Lock 139121486146512 released on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/106dc06a09bae48ef4f4a69b9510e2d7bce9ba6536728f70ca779cdf1b2f6db3.lock
2025-02-24 15:31:55,225 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /deepset/roberta-base-squad2-distilled/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-02-24 15:31:55,227 - DEBUG - filelock - Attempting to acquire lock 139121485557200 on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/6fe1dd378028ed71bba0026ca663cba4bba44438.lock
2025-02-24 15:31:55,229 - DEBUG - filelock - Lock 139121485557200 acquired on /root/.cache/huggingface/hub/.locks/models--d

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2025-02-24 15:31:58,095 - DEBUG - filelock - Attempting to release lock 139119827762576 on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/75801df688e89a96f642b98cbdd97288f5207518.lock
2025-02-24 15:31:58,098 - DEBUG - filelock - Lock 139119827762576 released on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/75801df688e89a96f642b98cbdd97288f5207518.lock
2025-02-24 15:31:58,524 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /deepset/roberta-base-squad2-distilled/resolve/main/added_tokens.json HTTP/1.1" 404 0
2025-02-24 15:31:58,737 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /deepset/roberta-base-squad2-distilled/resolve/main/special_tokens_map.json HTTP/1.1" 200 0
2025-02-24 15:31:58,739 - DEBUG - filelock - Attempting to acquire lock 139119829615824 on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/2ea7ad0e45a9d1d1591782ba7e29a703d0758

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

2025-02-24 15:31:58,967 - DEBUG - filelock - Attempting to release lock 139119829615824 on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/2ea7ad0e45a9d1d1591782ba7e29a703d0758831.lock
2025-02-24 15:31:58,968 - DEBUG - filelock - Lock 139119829615824 released on /root/.cache/huggingface/hub/.locks/models--deepset--roberta-base-squad2-distilled/2ea7ad0e45a9d1d1591782ba7e29a703d0758831.lock
2025-02-24 15:31:59,159 - DEBUG - urllib3.connectionpool - https://huggingface.co:443 "HEAD /deepset/roberta-base-squad2-distilled/resolve/main/chat_template.jinja HTTP/1.1" 404 0
2025-02-24 15:31:59,355 - DEBUG - haystack.core.pipeline.base - Adding component 'retriever' (<haystack.components.retrievers.in_memory.bm25_retriever.InMemoryBM25Retriever object at 0x7e87bf1c0d90>

Inputs:
  - query: str
  - filters: Optional[Dict[str, Any]]
  - top_k: Optional[int]
  - scale_score: Optional[bool]
Outputs:
  - documents: List[Document])
2025-02-24 15:31:59,363 - DEBUG - h

Log entries saved to log.txt
